In [7]:
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# -----------------------------
# CONFIGURATION
# -----------------------------
API_KEY = "YOUR_API_KEY"  # <-- Replace this
CITY = "Mumbai"
UNITS = "metric"
URL = f"http://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={API_KEY}&units={UNITS}"

# -----------------------------
# FETCH WEATHER DATA
# -----------------------------
response = requests.get(URL)
data = response.json()

# -----------------------------
# CHECK FOR ERRORS
# -----------------------------
if response.status_code != 200 or 'list' not in data:
    print("\n❌ Failed to fetch data:")
    print("Status Code:", response.status_code)
    print("Response:", data)
    exit()

# -----------------------------
# PARSE WEATHER DATA
# -----------------------------
dates = []
temperatures = []
humidities = []

for entry in data['list']:
    dt_txt = entry['dt_txt']
    temp = entry['main']['temp']
    humidity = entry['main']['humidity']

    dates.append(datetime.strptime(dt_txt, '%Y-%m-%d %H:%M:%S'))
    temperatures.append(temp)
    humidities.append(humidity)

# -----------------------------
# VISUALIZATION
# -----------------------------
plt.figure(figsize=(14, 6))
sns.lineplot(x=dates, y=temperatures, marker='o', color='tomato')
plt.title(f"5-Day Temperature Forecast for {CITY}", fontsize=16)
plt.xlabel("Date & Time")
plt.ylabel("Temperature (°C)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()



❌ Failed to fetch data:
Status Code: 401
Response: {'cod': 401, 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}


KeyError: 'list'

In [ ]:
import pandas as pd
from fpdf import FPDF

# Load the data
df = pd.read_csv("data.csv")

# Analysis
summary = df.groupby("Department")["Salary"].agg(["mean", "max", "min", "count"]).reset_index()

# PDF Setup
class PDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 14)
        self.cell(200, 10, "Employee Salary Report", ln=True, align="C")

    def footer(self):
        self.set_y(-15)
        self.set_font("Arial", "I", 8)
        self.cell(0, 10, f"Page {self.page_no()}", align="C")

    def chapter_title(self, title):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, title, ln=True)

    def add_summary_table(self, summary_df):
        self.set_font("Arial", "", 11)
        self.cell(0, 10, "", ln=True)  # spacing
        col_widths = [40, 30, 30, 30, 30]
        headers = ["Department", "Avg Salary", "Max Salary", "Min Salary", "Count"]

        # Header Row
        for i, header in enumerate(headers):
            self.cell(col_widths[i], 10, header, border=1, align="C")
        self.ln()

        # Data Rows
        for row in summary_df.itertuples(index=False):
            self.cell(col_widths[0], 10, str(row[0]), border=1)
            self.cell(col_widths[1], 10, f"{row[1]:.2f}", border=1)
            self.cell(col_widths[2], 10, f"{row[2]}", border=1)
            self.cell(col_widths[3], 10, f"{row[3]}", border=1)
            self.cell(col_widths[4], 10, f"{row[4]}", border=1)
            self.ln()

# Create PDF
pdf = PDF()
pdf.add_page()
pdf.chapter_title("Department-wise Salary Summary")
pdf.add_summary_table(summary)
pdf.output("Employee_Salary_Report.pdf")

print("✅ PDF report generated: Employee_Salary_Report.pdf")


In [ ]:
import nltk
import numpy as np
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Sample corpus (You can expand this or load from a file)
corpus = """
Hello, how can I help you?
What is your name?
I am a chatbot built using Python.
Tell me about natural language processing.
NLP stands for Natural Language Processing.
It is a field of AI that deals with human language.
What can you do?
I can answer questions, perform small talk, and help you with basic queries.
Goodbye!
"""

# Tokenize sentences
sent_tokens = nltk.sent_tokenize(corpus.lower())

# Lemmatizer
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# GREETING INPUTS AND RESPONSES
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
GREETING_RESPONSES = ["Hi there!", "Hey!", "Hello!", "Greetings!", "Hi! How can I assist?"]

def greet(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

# RESPONSE GENERATION
def generate_response(user_input):
    robo_response = ''
    sent_tokens.append(user_input)

    # Vectorization
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)

    # Compute similarity
    vals = cosine_similarity(tfidf[-1], tfidf[:-1])
    idx = np.argmax(vals)
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    # Check confidence
    if req_tfidf == 0:
        robo_response = "I'm sorry, I didn't understand that."
    else:
        robo_response = sent_tokens[idx]

    sent_tokens.pop()  # remove the last input to avoid growth
    return robo_response

# MAIN CHAT LOOP
def chatbot():
    print("BOT: Hello! I am your chatbot. Type 'bye' to exit.")
    while True:
        user_input = input("YOU: ").lower()
        if user_input in ['bye', 'exit', 'quit']:
            print("BOT: Goodbye! Have a great day.")
            break
        elif greet(user_input) is not None:
            print("BOT:", greet(user_input))
        else:
            print("BOT:", generate_response(user_input))

# Run the chatbot
if __name__ == "__main__":
    chatbot()


In [ ]:
Hello, how can I help you?
What is your name?
I am a chatbot built using Python.
Tell me about natural language processing.
NLP stands for Natural Language Processing.
It is a field of AI that deals with human language.
What can you do?
I can answer questions, perform small talk, and help you with basic queries.
Goodbye!


In [ ]:
# TASK - 4 : MACHINE LEARNING MODEL IMPLEMENTATION
# Example: Spam Email Detection using Scikit-learn

# Step 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Step 2: Load Dataset
# Using a sample dataset (you can replace with any spam dataset e.g. from Kaggle)
data = pd.read_csv("https://raw.githubusercontent.com/justmarkham/scikit-learn-videos/master/data/sms.tsv",
                   sep='\t', names=['label', 'message'])

print("Dataset Sample:")
print(data.head())

# Step 3: Preprocess Data
X = data['message']
y = data['label'].map({'ham': 0, 'spam': 1})   # Convert labels: ham=0, spam=1

# Convert text into numerical vectors
vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X)

# Step 4: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.3, random_state=42)

# Step 5: Train Model
model = MultinomialNB()
model.fit(X_train, y_train)

# Step 6: Predictions
y_pred = model.predict(X_test)

# Step 7: Evaluation
print("\nModel Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Step 8: Test with custom input
sample_msg = ["Congratulations! You've won a free ticket, claim now!",
              "Hey, are we meeting tomorrow at 5?"]

sample_msg_vec = vectorizer.transform(sample_msg)
pred = model.predict(sample_msg_vec)

print("\nSample Predictions:")
for msg, label in zip(sample_msg, pred):
    print(f"Message: {msg} --> {'SPAM' if label==1 else 'HAM'}")
